In [37]:
import csv
import re

def normalized(string):
    return re.sub(r'\s+', ' ', string.strip())

def convert_annotation(score):
    """
     This function converts annotations to numerical values:
     negative - 0, positive - 1, inclonclusive - 2
    """ 
    if score in ['p','p1','p2','p3']:
        score = '1'
    elif score in ['n1','n2','n3']:
        score = '0'
    elif score == 'inc':
        score = '2'
    else:
        print("score is malformed")

    return score

def get_scores_dict(filename):

    filedict = {}
    
    with open(f"../experiment_1/anthroscore/expectations/{filename}.csv","r") as csv_file:        
        header = csv_file.readline()
        reader = csv.reader(csv_file)
        for row in reader:
            sentence_id = row[0]
            sentence_info = row[1:]
            filedict[sentence_id] = sentence_info # IDs are unique

    return filedict

def concat_info(filename,filedict):

    with open(f"../experiment_1/anthroscore/predictions/expanded_sentence_scores/{filename}.csv","w") as outfile:

        list_check = []
        
        writer = csv.writer(outfile)
        new_header = ['id','sentence','masked_sentence','AI_phrase','suggested_mask','AI_entity',
                        'anthro_component','original_term','original_noun','expectation','anthroscore']
        writer.writerow(new_header)
        infile = open(f"../experiment_1/anthroscore/predictions/sentence_scores/{filename}.csv","r")
        header = infile.readline()
        reader = csv.reader(infile)
        
        for row in reader:
            
            sentence_id = normalized(row[3])
            sentence = normalized(row[1])
            masked = normalized(row[2])
            original_term = normalized(row[6])
            original_noun = normalized(row[7])
            anthroscore = normalized(row[8])
            info = [normalized(x) for x in filedict[sentence_id]]
            orig_score = convert_annotation(info[-1])

            write_to_file = [sentence_id,sentence,masked]+info[1:-1]+[original_term,original_noun,orig_score,anthroscore]

            writer.writerow(write_to_file)


for file in files:
    file_dict = get_scores_dict(file)
    concat_info(file,file_dict)

In [3]:
import pandas as pd

files = ["adjective_phrases_inconclusive",
         "adjective_phrases_negative",
         "adjective_phrases_positive",
         "comparisons_inconclusive",
         "noun_phrases_positive",
         "possessives_positive",
         "verb_objects_inconclusive",
         "verb_objects_negative",
         "verb_objects_positive",
         "verb_subjects_inconclusive",
         "verb_subjects_negative",
         "verb_subjects_positive"
        ]

for file in files:
    df = pd.read_csv(f'../experiment_1/anthroscore/predictions/csv/{file}.csv')  # comma is default
    df.to_csv(
        f'../experiment_1/anthroscore/predictions/txt/{file}.txt',
        sep='\t',
        index=False,
        header=False
    )